# LangChain 实战：iPhone销售聊天机器人

## 使用 GPT 构造销售话术数据

ChatGPT 分享链接：https://chatgpt.com/share/fdadd68e-ed6f-4ea2-b5b1-4dc97f282c6b


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是库克手下最牛的iPhone手机销售，现在培训职场新人，请给出100条实用的销售话术帮助用户自适应。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT 回复结果：
![](../../snapshots/iphone_sales_data.png)

## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [50]:
with open("real_estate_sales_data.txt", "r", encoding="utf-8") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [51]:
from langchain.text_splitter import CharacterTextSplitter

In [52]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [53]:
docs = text_splitter.create_documents([real_estate_sales])

In [54]:
docs[0]

Document(page_content='[客户问题]：这款iPhone和上一代相比有什么区别？\n[销售回答]：新的iPhone在处理器性能、摄像头功能和电池续航方面都有显著提升，特别是在拍照和视频录制方面，体验会更好。')

In [55]:
len(docs)

71

### 使用 Faiss 作为向量数据库，持久化存储iPhone销售 问答对（QA-Pair）

In [56]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [57]:
query = "iPhone如何保修？"

In [58]:
answer_list = db.similarity_search(query)

In [59]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]：iPhone的售后服务怎么样？
[销售回答]：苹果提供一年的有限保修服务和90天的免费技术支持，您还可以购买AppleCare+来延长保修时间和享受更多服务。

[客户问题]：iPhone的售后政策是什么？
[销售回答]：苹果提供一年的有限保修服务和90天的免费技术支持，您还可以购买AppleCare+来延长保修时间和享受更多服务。

[客户问题]：iPhone的防摔性能怎么样？
[销售回答]：这款iPhone采用了超瓷晶面板，防摔性能比普通玻璃提升了四倍，更加耐用。

[客户问题]：iPhone的售后网点多吗？
[销售回答]：苹果的售后服务网点遍布全国，您可以在任何一家苹果授权服务提供商处享受专业的售后服务。



In [60]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [61]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [62]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002532C2B6FE0>, search_kwargs={'k': 3})

In [63]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：iPhone的售后服务怎么样？
[销售回答]：苹果提供一年的有限保修服务和90天的免费技术支持，您还可以购买AppleCare+来延长保修时间和享受更多服务。

[客户问题]：iPhone的售后政策是什么？
[销售回答]：苹果提供一年的有限保修服务和90天的免费技术支持，您还可以购买AppleCare+来延长保修时间和享受更多服务。

[客户问题]：iPhone的防摔性能怎么样？
[销售回答]：这款iPhone采用了超瓷晶面板，防摔性能比普通玻璃提升了四倍，更加耐用。



In [64]:
docs = topK_retriever.get_relevant_documents("新的iPhone的发售时间")

In [65]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：iPhone的续航时间有多长？
[销售回答]：这款iPhone的电池续航经过优化，可以满足您一整天的使用需求，包括打电话、上网、拍照等日常操作。

[客户问题]：这款iPhone的刷新率是多少？
[销售回答]：这款iPhone的屏幕刷新率为60Hz，但高端型号支持120Hz ProMotion技术，显示效果更加流畅。

[客户问题]：购买这款iPhone有没有优惠活动？
[销售回答]：目前我们有一些优惠活动，比如以旧换新和分期付款，具体优惠您可以在我们店内了解详情。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [66]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [67]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：iPhone的售后服务怎么样？
[销售回答]：苹果提供一年的有限保修服务和90天的免费技术支持，您还可以购买AppleCare+来延长保修时间和享受更多服务。

[客户问题]：iPhone的售后政策是什么？
[销售回答]：苹果提供一年的有限保修服务和90天的免费技术支持，您还可以购买AppleCare+来延长保修时间和享受更多服务。



### 提取向量数据库中的`销售回答`

In [68]:
docs = retriever.get_relevant_documents(query)

In [69]:
docs[0].page_content

'[客户问题]：iPhone的售后服务怎么样？\n[销售回答]：苹果提供一年的有限保修服务和90天的免费技术支持，您还可以购买AppleCare+来延长保修时间和享受更多服务。'

In [70]:
docs[0].page_content.split("[销售回答]：")

['[客户问题]：iPhone的售后服务怎么样？\n',
 '苹果提供一年的有限保修服务和90天的免费技术支持，您还可以购买AppleCare+来延长保修时间和享受更多服务。']

In [71]:
ans = docs[0].page_content.split("[销售回答]：")[-1]

In [72]:
ans

'苹果提供一年的有限保修服务和90天的免费技术支持，您还可以购买AppleCare+来延长保修时间和享受更多服务。'

#### 尝试各种问题

In [73]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答]：")[-1] for doc in docs]

    return ans_list

In [74]:
query = "我想要潜水，可以在水下使用吗"

print(sales(query))

[]


c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [75]:
print(sales(query, 0.5))

['是的，这款iPhone具备防水性能，可以在1米深的水中停留30分钟，非常适合日常使用中的意外情况。', '这款iPhone的电池续航时间经过优化，可以满足您一整天的使用需求，包括打电话、上网、拍照等日常操作。', '是的，iPhone内置了Siri语音助手，您可以通过语音指令控制手机，非常方便。', '目前这款iPhone主要使用面容ID进行解锁，面容ID的安全性和便捷性都非常高。']


In [76]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['iPhone的保值率一直很高，二手市场价格也非常坚挺，您可以放心购买。', '这款iPhone配备了1200万像素的后置摄像头，前置摄像头也有1200万像素，拍照效果非常出色。', '目前我们有一些优惠活动，比如以旧换新和分期付款，具体优惠您可以在我们店内了解详情。', 'iPhone虽然价格稍高，但在硬件、软件、生态系统和售后服务等方面都非常出色，性价比非常高。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [93]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.75}))

In [78]:
qa_chain({"query": "我想要潜水，可以在水下使用iPhone吗"})

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想要潜水，可以在水下使用iPhone吗',
 'result': 'iPhone的某些型号具有一定的防水能力，但苹果公司的官方建议是不要在所有类型的水下活动或极端深度下使用iPhone。即使是防水等级较高的iPhone，也只是在特定条件下承诺防水，例如在一定的深度和时间内。潜水时的水压远远超出了苹果公司建议的使用条件。\n\n如果你想要在水下使用手机拍照或录像，应该购买一个专门为潜水设计的防水壳。这样的防水壳可以承受更大的深度和压力，保护你的iPhone免受水下环境的损害。在选择防水壳时，请确认其防水深度和质量，确保它能够满足你潜水时的需要。'}

In [94]:
qa_chain({"query": "iPhone是什么接口"})

{'query': 'iPhone是什么接口', 'result': 'iPhone使用的是Lightning接口。'}

In [95]:
print(sales("iPhone是什么接口", score_threshold=0.75))

['iPhone使用的是Lightning接口，传输速度非常快，而且还支持USB \\d+\\.0标准，数据传输更加高效。', 'FaceTime是iPhone内置的视频通话功能，通话质量非常清晰，可以让您和家人朋友随时保持联系。', 'iPhone的触控反应非常灵敏，屏幕操作非常流畅，能够快速响应您的每一个操作。', 'iPhone内置了强大的地图和导航功能，精确度非常高，能够快速带您找到目的地。']


## 加载 FAISS 向量数据库已有结果

In [96]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [98]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [99]:
qa_chain({"query": "我想买iPhone，有什么渠道"})

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买iPhone，有什么渠道',
 'result': '您可以通过以下渠道购买iPhone：\n\n1. Apple官方网站或实体店：访问苹果官方网站或前往最近的苹果零售店购买。\n\n2. 授权经销商：前往您所在地区的授权经销商购买，例如官方授权的电子产品商店。\n\n3. 电商平台：在诸如京东、天猫、苏宁易购等大型电商平台购买。\n\n4. 运营商：与您的移动运营商合作购买，例如中国移动、中国联通和中国电信等。\n\n请确保购买的iPhone是正品，并在可信任的渠道购买以获得售后保障。'}

In [100]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [101]:
qa_chain({"query": "我想买iPhone，有什么渠道"})

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买iPhone，有什么渠道',
 'result': '你可以通过多种渠道购买iPhone。例如，你可以直接在Apple的官方网站或实体店购买。另外，你也可以在各大电子产品零售商，如亚马逊、京东、苏宁易购等网站购买。此外，还可以在各大电信运营商购买，如中国移动、中国联通和中国电信等。请确保你购买的是正品，并且在购买时了解清楚售后服务政策。'}

In [ ]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [104]:
result = qa_chain({"query": "我想买iPhone，有什么渠道"})

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [105]:
result

{'query': '我想买iPhone，有什么渠道',
 'result': '购买iPhone有以下几种渠道：\n\n1. Apple官方商店：您可以在Apple官方网站或实体店购买iPhone，这是最直接的渠道，可以确保手机的品质和售后服务。\n\n2. 运营商：您可以通过电信运营商购买iPhone，如中国移动、中国联通、中国电信等。这样购买手机可能会有一些优惠套餐，但需要注意合约细则。\n\n3. 电子商务平台：您还可以在一些知名电商平台购买iPhone，如京东、天猫、苏宁易购等。购买时请注意选择正品授权经销商，以确保手机品质。\n\n4. 二手市场：如果您不介意购买二手手机，可以在闲鱼、转转等二手交易平台购买iPhone。购买二手手机时要注意检查手机的外观、功能和保修情况。\n\n无论选择哪种渠道，请务必注意购买正品，并确保有完善的售后保障。',
 'source_documents': []}